In [2]:
import pandas as pd
import glob
import os

# 📁 Thư mục chứa các file dữ liệu
input_dir = "data_raw"      # đổi lại đúng thư mục bạn để file
output_file = "merged_restaurants.csv"

# 🧩 Lấy danh sách tất cả file .csv hoặc .tsv
files = glob.glob(os.path.join(input_dir, "*.csv")) + glob.glob(os.path.join(input_dir, "*.tsv"))

all_dfs = []

for f in files:
    try:
        # Auto detect delimiter (vì Google export có thể "," hoặc tab)
        df = pd.read_csv(f, sep=None, engine='python', on_bad_lines='skip')
        df["source_file"] = os.path.basename(f)   # thêm cột nguồn để trace dữ liệu
        all_dfs.append(df)
        print(f"✅ Loaded: {f} - {len(df)} rows")
    except Exception as e:
        print(f"❌ Error loading {f}: {e}")

# 📊 Gộp tất cả file
if all_dfs:
    merged_df = pd.concat(all_dfs, ignore_index=True)

    # 🧹 Làm sạch: bỏ hàng rỗng hoàn toàn
    merged_df.dropna(how="all", inplace=True)

    # 🧾 Chuẩn tên cột (loại bỏ ký tự đặc biệt, viết thường)
    merged_df.columns = [
        c.strip().lower().replace(" ", "_").replace(".", "").replace("-", "_")
        for c in merged_df.columns
    ]

    # 🧩 Bỏ cột trùng
    merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

    # 💾 Xuất ra CSV
    merged_df.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"\n🎉 Merged {len(files)} files -> {output_file} ({len(merged_df)} rows)")
else:
    print("⚠️ Không có file nào được load.")


✅ Loaded: data_raw\google (1).csv - 175 rows
✅ Loaded: data_raw\google (10).csv - 243 rows
✅ Loaded: data_raw\google (11).csv - 5612 rows
✅ Loaded: data_raw\google (12).csv - 4412 rows
✅ Loaded: data_raw\google (2).csv - 236 rows
✅ Loaded: data_raw\google (3).csv - 126 rows
✅ Loaded: data_raw\google (4).csv - 55 rows
✅ Loaded: data_raw\google (5).csv - 209 rows
✅ Loaded: data_raw\google (6).csv - 209 rows
✅ Loaded: data_raw\google (7).csv - 727 rows
✅ Loaded: data_raw\google (8).csv - 2715 rows
✅ Loaded: data_raw\google (9).csv - 16 rows

🎉 Merged 12 files -> merged_restaurants.csv (14735 rows)


In [4]:
import pandas as pd

# === ĐƯỜNG DẪN FILE NGUỒN ===
input_file = "merged_data.csv"      # File gốc bạn đã gộp
output_file = "href_links.csv"      # File xuất chỉ chứa link

# === ĐỌC FILE GỐC ===
df = pd.read_csv(input_file)

# === KIỂM TRA CỘT ===
target_col = None
for col in df.columns:
    if col.lower() in ["hfpxzc_href", "href", "link"]:
        target_col = col
        break

if target_col is None:
    print("❌ Không tìm thấy cột hfpxzc_href hoặc href trong file!")
else:
    # === LẤY CỘT VÀ LOẠI GIÁ TRỊ RỖNG ===
    href_df = df[[target_col]].dropna().drop_duplicates()
    href_df.columns = ["Link"]  # Đổi tên cột cho đẹp

    # === XUẤT RA FILE MỚI ===
    href_df.to_csv(output_file, index=False, encoding="utf-8-sig")

    print(f"✅ Đã trích xuất {len(href_df)} link vào: {output_file}")


✅ Đã trích xuất 12271 link vào: href_links.csv
